In [1]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
# import glove nitin commented as its not used
# from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, merge

# from keras.optimizers import Adam # nitin commented as Adam has been shifted to optimizer_v1 module.
from tensorflow.keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
#from keras.backend.tensorflow_backend import set_session, clear_session, get_session # nitin commented as tensorflow_backend not used anymore
from keras.backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
def create_dataset(dict_of_ner):
    temp_data = []
    for k, v in sorted(dict_of_ner.items()):
        temp = []
        for embed in v:
            temp.append(embed)
            #print(embed)
        temp_data.append(np.mean(temp, axis = 0)) 
    return np.asarray(temp_data)

def make_prediction_multi_avg(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_multi_avg(predictions, probs, ground_truth, 
                          
                          embed_name, problem_type, iteration, hidden_unit_size,
                          
                          sequence_name, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1
    
    result_path = "results/"
    file_name = str(sequence_name)+"-"+str(hidden_unit_size)+"-"+embed_name
    file_name = file_name +"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+"-avg-.p"
    
    print('auc{0}, auprc{1}, acc{2}, F1{3} '.format(auc,auprc,acc,F1))
    
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)
    
def avg_ner_model(layer_name, number_of_unit, embedding_name):

    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    sequence_input = Input(shape=(24,104))

    input_avg = Input(shape=(input_dimension, ), name = "avg")        
#     x_1 = Dense(256, activation='relu')(input_avg)
#     x_1 = Dropout(0.3)(x_1)
    
    if layer_name == "GRU":
        x = GRU(number_of_unit)(sequence_input)
    elif layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)

    x = keras.layers.Concatenate()([x, input_avg])

    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    
    #logits_regularizer = tf.contrib.layers.l2_regularizer(scale=0.01) nitin commented as its deprecated
    logits_regularizer = tf.keras.regularizers.L2(0.01)
    # removed tf.compat.v1.estimator.layers.xavier_initializer() with glorot_normal
    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.glorot_normal(), 
                  kernel_regularizer=logits_regularizer)(x)
    
    
    opt = Adam(lr=0.001, decay = 0.01)
    model = Model(inputs=[sequence_input, input_avg], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['acc'])
    
    return model

In [3]:
# nitin added as without it the read_pickel below was giving the error as in notebook 5 when its writing
#pickel do a lazy execution and write the function as well
def mean(a):
    return sum(a) / len(a)

In [4]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")

ner_word2vec = pd.read_pickle("data/"+type_of_ner+"_ner_word2vec_limited_dict.pkl")
ner_fasttext = pd.read_pickle("data/"+type_of_ner+"_ner_fasttext_limited_dict.pkl")
ner_concat = pd.read_pickle("data/"+type_of_ner+"_ner_combined_limited_dict.pkl")

train_ids = pd.read_pickle("data/"+type_of_ner+"_train_ids.pkl")
dev_ids = pd.read_pickle("data/"+type_of_ner+"_dev_ids.pkl")
test_ids = pd.read_pickle("data/"+type_of_ner+"_test_ids.pkl")

from collections.abc import Iterable, Iterator
def create_dataset(dict_of_ner):
    temp_data = []
    for k, v in sorted(dict_of_ner.items()):
        temp = []
        for embed in v:
            print(type(embed),embed)
            print(isinstance(embed, Iterator))
            temp.append(embed)
            
        temp_data.append(np.mean(temp, axis = 0)) 
    return np.asarray(temp_data)

tt = dict({ke:va})
create_dataset(tt)

for ke, va in temp_train_ner.items():
    print(ke, va)
    print(nit)

In [5]:
embedding_types = ['word2vec', 'fasttext']#, 'concat']
embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]
target_problems = ['mort_hosp', 'mort_icu']#, 'los_3', 'los_7']


num_epoch = 100
model_patience = 5
monitor_criteria = 'val_loss'
batch_size = 64
iter_num = 11
unit_sizes = [128, 256]

#layers = ["LSTM", "GRU"]
layers = ["GRU"]
for each_layer in layers:
    print ("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print ("Hidden unit: ", each_unit_size)

        for embed_dict, embed_name in zip(embedding_dict, embedding_types):    
            print ("Embedding: ", embed_name)
            print("=============================")
            
            temp_train_ner = dict((k, ner_word2vec[k]) for k in train_ids)
            temp_dev_ner = dict((k, ner_word2vec[k]) for k in dev_ids)
            temp_test_ner = dict((k, ner_word2vec[k]) for k in test_ids)

            x_train_ner = create_dataset(temp_train_ner)
            x_dev_ner = create_dataset(temp_dev_ner)
            x_test_ner = create_dataset(temp_test_ner)


            for iteration in range(1, iter_num):
                print ("Iteration number: ", iteration)

                for each_problem in target_problems:
                    print ("Problem type: ", each_problem)
                    print ("__________________")

                    early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                    best_model_name = "avg-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                    checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                        save_best_only=True, mode='min', period=1)


                    callbacks = [early_stopping_monitor, checkpoint]

                    model = avg_ner_model(each_layer, each_unit_size, embed_name)
                    
                    model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1, 
                              validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, 
                              batch_size=batch_size )

                    model.load_weights(best_model_name)

                    probs, predictions = make_prediction_multi_avg(model, [x_test_lstm, x_test_ner])
                    
                    # nitin uncomment the code
                    save_scores_multi_avg(predictions, probs, y_test[each_problem], 
                               embed_name, each_problem, iteration, each_unit_size, 
                               each_layer, type_of_ner)
                    
                    #reset_keras(model) nitin commented as its not defined
                    #del model
                    sess = get_session()
                    clear_session()
                    sess.close()
                    #clear_session() nitin originally its only clear session but added abve three line
                    gc.collect()

Layer:  GRU
Hidden unit:  128
Embedding:  word2vec
Iteration number:  1
Problem type:  mort_hosp
__________________
Epoch 1/100
238/238 [==============================] - ETA: 0s - loss: 0.3027 - acc: 0.8992
Epoch 00001: val_loss improved from inf to 0.24437, saving model to avg-word2vec-mort_hosp-best_model.hdf5
238/238 [==============================] - 17s 48ms/step - loss: 0.3027 - acc: 0.8992 - val_loss: 0.2444 - val_acc: 0.9168
Epoch 2/100
238/238 [==============================] - ETA: 0s - loss: 0.2531 - acc: 0.9137
Epoch 00002: val_loss improved from 0.24437 to 0.24135, saving model to avg-word2vec-mort_hosp-best_model.hdf5
238/238 [==============================] - 10s 41ms/step - loss: 0.2531 - acc: 0.9137 - val_loss: 0.2413 - val_acc: 0.9173
Epoch 3/100
237/238 [============================>.] - ETA: 0s - loss: 0.2386 - acc: 0.9172
Epoch 00003: val_loss improved from 0.24135 to 0.23650, saving model to avg-word2vec-mort_hosp-best_model.hdf5
238/238 [========================

238/238 [==============================] - ETA: 0s - loss: 0.1456 - acc: 0.9502
Epoch 00016: val_loss did not improve from 0.17046
238/238 [==============================] - 11s 45ms/step - loss: 0.1456 - acc: 0.9502 - val_loss: 0.1714 - val_acc: 0.9404
Epoch 17/100
237/238 [============================>.] - ETA: 0s - loss: 0.1463 - acc: 0.9513
Epoch 00017: val_loss did not improve from 0.17046
238/238 [==============================] - 10s 44ms/step - loss: 0.1462 - acc: 0.9513 - val_loss: 0.1708 - val_acc: 0.9404
Epoch 18/100
237/238 [============================>.] - ETA: 0s - loss: 0.1448 - acc: 0.9518
Epoch 00018: val_loss did not improve from 0.17046
238/238 [==============================] - 11s 46ms/step - loss: 0.1447 - acc: 0.9519 - val_loss: 0.1711 - val_acc: 0.9422
Epoch 19/100
238/238 [==============================] - ETA: 0s - loss: 0.1441 - acc: 0.9505
Epoch 00019: val_loss did not improve from 0.17046
238/238 [==============================] - 10s 44ms/step - loss: 0.1

Problem type:  mort_icu
__________________
Epoch 1/100
238/238 [==============================] - ETA: 0s - loss: 0.2244 - acc: 0.9323
Epoch 00001: val_loss improved from inf to 0.18150, saving model to avg-word2vec-mort_icu-best_model.hdf5
238/238 [==============================] - 17s 50ms/step - loss: 0.2244 - acc: 0.9323 - val_loss: 0.1815 - val_acc: 0.9432
Epoch 2/100
237/238 [============================>.] - ETA: 0s - loss: 0.1846 - acc: 0.9418
Epoch 00002: val_loss improved from 0.18150 to 0.17649, saving model to avg-word2vec-mort_icu-best_model.hdf5
238/238 [==============================] - 10s 43ms/step - loss: 0.1846 - acc: 0.9418 - val_loss: 0.1765 - val_acc: 0.9422
Epoch 3/100
237/238 [============================>.] - ETA: 0s - loss: 0.1739 - acc: 0.9431
Epoch 00003: val_loss improved from 0.17649 to 0.17581, saving model to avg-word2vec-mort_icu-best_model.hdf5
238/238 [==============================] - 11s 44ms/step - loss: 0.1740 - acc: 0.9432 - val_loss: 0.1758 - va

237/238 [============================>.] - ETA: 0s - loss: 0.1868 - acc: 0.9403
Epoch 00002: val_loss improved from 0.18448 to 0.18087, saving model to avg-word2vec-mort_icu-best_model.hdf5
238/238 [==============================] - 11s 45ms/step - loss: 0.1868 - acc: 0.9403 - val_loss: 0.1809 - val_acc: 0.9413
Epoch 3/100
238/238 [==============================] - ETA: 0s - loss: 0.1772 - acc: 0.9430
Epoch 00003: val_loss improved from 0.18087 to 0.17781, saving model to avg-word2vec-mort_icu-best_model.hdf5
238/238 [==============================] - 10s 44ms/step - loss: 0.1772 - acc: 0.9430 - val_loss: 0.1778 - val_acc: 0.9395
Epoch 4/100
238/238 [==============================] - ETA: 0s - loss: 0.1701 - acc: 0.9451
Epoch 00004: val_loss improved from 0.17781 to 0.17677, saving model to avg-word2vec-mort_icu-best_model.hdf5
238/238 [==============================] - 10s 41ms/step - loss: 0.1701 - acc: 0.9451 - val_loss: 0.1768 - val_acc: 0.9409
Epoch 5/100
237/238 [================

237/238 [============================>.] - ETA: 0s - loss: 0.2354 - acc: 0.9175
Epoch 00004: val_loss improved from 0.24224 to 0.24205, saving model to avg-word2vec-mort_hosp-best_model.hdf5
238/238 [==============================] - 9s 39ms/step - loss: 0.2357 - acc: 0.9174 - val_loss: 0.2421 - val_acc: 0.9136
Epoch 5/100
237/238 [============================>.] - ETA: 0s - loss: 0.2284 - acc: 0.9188
Epoch 00005: val_loss improved from 0.24205 to 0.24135, saving model to avg-word2vec-mort_hosp-best_model.hdf5
238/238 [==============================] - 9s 39ms/step - loss: 0.2288 - acc: 0.9186 - val_loss: 0.2414 - val_acc: 0.9140
Epoch 6/100
238/238 [==============================] - ETA: 0s - loss: 0.2260 - acc: 0.9209
Epoch 00006: val_loss did not improve from 0.24135
238/238 [==============================] - 9s 39ms/step - loss: 0.2260 - acc: 0.9209 - val_loss: 0.2425 - val_acc: 0.9122
Epoch 7/100
238/238 [==============================] - ETA: 0s - loss: 0.2246 - acc: 0.9205
Epoch

...

237/238 [============================>.] - ETA: 0s - loss: 0.1809 - acc: 0.9440- ETA: 0s - loss: 0.1812 - acc: 0.943
Epoch 00002: val_loss improved from 0.18242 to 0.17693, saving model to avg-fasttext-mort_icu-best_model.hdf5
238/238 [==============================] - 11s 46ms/step - loss: 0.1811 - acc: 0.9439 - val_loss: 0.1769 - val_acc: 0.9432
Epoch 3/100
238/238 [==============================] - ETA: 0s - loss: 0.1690 - acc: 0.9460
Epoch 00003: val_loss improved from 0.17693 to 0.17621, saving model to avg-fasttext-mort_icu-best_model.hdf5
238/238 [==============================] - 11s 46ms/step - loss: 0.1690 - acc: 0.9460 - val_loss: 0.1762 - val_acc: 0.9427
Epoch 4/100
237/238 [============================>.] - ETA: 0s - loss: 0.1626 - acc: 0.9463
Epoch 00004: val_loss improved from 0.17621 to 0.17547, saving model to avg-fasttext-mort_icu-best_model.hdf5
238/238 [==============================] - 11s 46ms/step - loss: 0.1624 - acc: 0.9464 - val_loss: 0.1755 - val_acc: 0.9418


238/238 [==============================] - ETA: 0s - loss: 0.2113 - acc: 0.9357
Epoch 00001: val_loss improved from inf to 0.17859, saving model to avg-fasttext-mort_icu-best_model.hdf5
238/238 [==============================] - 14s 49ms/step - loss: 0.2113 - acc: 0.9357 - val_loss: 0.1786 - val_acc: 0.9404
Epoch 2/100
237/238 [============================>.] - ETA: 0s - loss: 0.1771 - acc: 0.9446
Epoch 00002: val_loss improved from 0.17859 to 0.17248, saving model to avg-fasttext-mort_icu-best_model.hdf5
238/238 [==============================] - 12s 49ms/step - loss: 0.1767 - acc: 0.9447 - val_loss: 0.1725 - val_acc: 0.9450
Epoch 3/100
237/238 [============================>.] - ETA: 0s - loss: 0.1669 - acc: 0.9462- ETA: 6s - 
Epoch 00003: val_loss improved from 0.17248 to 0.17208, saving model to avg-fasttext-mort_icu-best_model.hdf5
238/238 [==============================] - 11s 47ms/step - loss: 0.1665 - acc: 0.9464 - val_loss: 0.1721 - val_acc: 0.9404
Epoch 4/100
237/238 [========

Epoch 7/100
238/238 [==============================] - ETA: 0s - loss: 0.1477 - acc: 0.9521
Epoch 00007: val_loss did not improve from 0.17119
238/238 [==============================] - 11s 47ms/step - loss: 0.1477 - acc: 0.9521 - val_loss: 0.1726 - val_acc: 0.9413
Epoch 8/100
238/238 [==============================] - ETA: 0s - loss: 0.1461 - acc: 0.9520- ETA:
Epoch 00008: val_loss did not improve from 0.17119
238/238 [==============================] - 11s 47ms/step - loss: 0.1461 - acc: 0.9520 - val_loss: 0.1723 - val_acc: 0.9418
Epoch 9/100
238/238 [==============================] - ETA: 0s - loss: 0.1441 - acc: 0.9524- E
Epoch 00009: val_loss did not improve from 0.17119
238/238 [==============================] - 11s 46ms/step - loss: 0.1441 - acc: 0.9524 - val_loss: 0.1734 - val_acc: 0.9404
Epoch 10/100
238/238 [==============================] - ETA: 0s - loss: 0.1407 - acc: 0.9531
Epoch 00010: val_loss did not improve from 0.17119
238/238 [==============================] - 12s 51

237/238 [============================>.] - ETA: 0s - loss: 0.1375 - acc: 0.9537
Epoch 00012: val_loss did not improve from 0.17016
238/238 [==============================] - 12s 49ms/step - loss: 0.1375 - acc: 0.9537 - val_loss: 0.1717 - val_acc: 0.9418
auc0.8899745618552679, auprc0.5289451980821587, acc0.9431922723091076, F10.4595185995623632 
0.8899745618552679 0.5289451980821587 0.9431922723091076 0.4595185995623632
Iteration number:  9
Problem type:  mort_hosp
__________________
Epoch 1/100
237/238 [============================>.] - ETA: 0s - loss: 0.3036 - acc: 0.8984- ETA: 6
Epoch 00001: val_loss improved from inf to 0.24768, saving model to avg-fasttext-mort_hosp-best_model.hdf5
238/238 [==============================] - 14s 50ms/step - loss: 0.3032 - acc: 0.8985 - val_loss: 0.2477 - val_acc: 0.9136
Epoch 2/100
238/238 [==============================] - ETA: 0s - loss: 0.2443 - acc: 0.9169
Epoch 00002: val_loss improved from 0.24768 to 0.24385, saving model to avg-fasttext-mort_

Epoch 3/100
238/238 [==============================] - ETA: 0s - loss: 0.2316 - acc: 0.9191- ETA: 2s - 
Epoch 00003: val_loss improved from 0.23751 to 0.23423, saving model to avg-fasttext-mort_hosp-best_model.hdf5
238/238 [==============================] - 12s 49ms/step - loss: 0.2316 - acc: 0.9191 - val_loss: 0.2342 - val_acc: 0.9164
Epoch 4/100
238/238 [==============================] - ETA: 0s - loss: 0.2242 - acc: 0.9214- ETA: 5s -
Epoch 00004: val_loss improved from 0.23423 to 0.23387, saving model to avg-fasttext-mort_hosp-best_model.hdf5
238/238 [==============================] - 11s 48ms/step - loss: 0.2242 - acc: 0.9214 - val_loss: 0.2339 - val_acc: 0.9191
Epoch 5/100
237/238 [============================>.] - ETA: 0s - loss: 0.2173 - acc: 0.9235- ETA
Epoch 00005: val_loss did not improve from 0.23387
238/238 [==============================] - 12s 49ms/step - loss: 0.2178 - acc: 0.9232 - val_loss: 0.2342 - val_acc: 0.9182
Epoch 6/100
238/238 [==============================] -